### summarize random forests results on Independent Simulation Scenarios for continuous outcome

In [1]:
dir = '/panfs/panfs1.ucsd.edu/panscratch/lij014/Stability_2020/sim_data'

load(paste0(dir, '/independent_RF.RData'))

In [2]:
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [3]:
avg_FDR = NULL
for (i in 1:length(files)){
    sim_file = files[i]
    load(sim_file, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_rf[[i]]$Stab.table
    
    FDR = NULL
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}

In [31]:
table_ind = NULL
tmp_num_select = rep(0, length(results_ind_rf))
for (i in 1:length(results_ind_rf)){
    results_ind_rf[[i]]$OOB = paste(round(mean(results_ind_rf[[i]]$OOB.list, na.rm=T),2),
                                    '(', round(FSA::se(results_ind_rf[[i]]$OOB.list, na.rm=T),2), ')')
    table_ind = rbind(table_ind, results_ind_rf[[i]][c('n', 'p', 'rou', 'FP', 'FN', 'MSE', 'Stab', 'OOB')])
    tmp_num_select[i] = mean(rowSums(results_ind_rf[[i]]$Stab.table))
}
table_ind = as.data.frame(table_ind)
table_ind$num_select = tmp_num_select
table_ind$FDR = round(avg_FDR,2)

In [32]:
head(table_ind)

n,p,rou,FP,FN,MSE,Stab,OOB,num_select,FDR
50,50,0,1 ( 0 ),6 ( 0 ),1.31 ( 0.06 ),NaN,2.09 ( 0.02 ),0.00,NaN
100,50,0,1.85 ( 0.11 ),4.11 ( 0.1 ),0.85 ( 0.02 ),0.18,1.79 ( 0.01 ),3.74,0.48
500,50,0,0.3 ( 0.05 ),1.61 ( 0.08 ),0.47 ( 0.01 ),0.8,1.31 ( 0 ),4.69,0.05
1000,50,0,0.07 ( 0.03 ),0.78 ( 0.07 ),0.35 ( 0 ),0.88,1.17 ( 0 ),5.29,0.01
50,100,0,1 ( 0 ),6 ( 0 ),1.32 ( 0.05 ),NaN,2.23 ( 0.02 ),0.00,NaN
100,100,0,4.33 ( 0.22 ),4.45 ( 0.09 ),0.91 ( 0.03 ),0.08,2.02 ( 0.01 ),5.88,0.72


In [33]:
# export result
result.table_ind <- apply(table_ind,2,as.character)
rownames(result.table_ind) = rownames(table_ind)
result.table_ind = as.data.frame(result.table_ind)

# extract numbers only for 'n' & 'p'
result.table_ind$n = tidyr::extract_numeric(result.table_ind$n)
result.table_ind$p = tidyr::extract_numeric(result.table_ind$p)
result.table_ind$ratio = result.table_ind$p / result.table_ind$n

result.table_ind = result.table_ind[c('n', 'p', 'ratio', 'Stab', 'MSE', 'FP', 'FN', 'OOB', 'num_select', 'FDR')]
colnames(result.table_ind)[1:3] = c('N', 'P', 'Ratio')
result.table_ind$Index = seq(1, length(results_ind_rf), 1)

extract_numeric() is deprecated: please use readr::parse_number() instead
extract_numeric() is deprecated: please use readr::parse_number() instead


In [34]:
# convert interested measurements to be numeric
result.table_ind$Stab = as.numeric(as.character(result.table_ind$Stab))
result.table_ind$MSE_mean = as.numeric(substr(result.table_ind$MSE, start=1, stop=4))
result.table_ind$FP_mean = as.numeric(substr(result.table_ind$FP, start=1, stop=4))
result.table_ind$FN_mean = as.numeric(substr(result.table_ind$FN, start=1, stop=4))
result.table_ind$OOB_mean = as.numeric(substr(result.table_ind$OOB, start=1, stop=4))
result.table_ind$num_select = as.numeric(as.character(result.table_ind$num_select))
#result.table_ind$FDR = as.numeric(as.character(result.table_ind$FDR))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [35]:
result.table_ind

N,P,Ratio,Stab,MSE,FP,FN,OOB,num_select,FDR,Index,MSE_mean,FP_mean,FN_mean,OOB_mean
50,50,1.00,NaN,1.31 ( 0.06 ),1 ( 0 ),6 ( 0 ),2.09 ( 0.02 ),0.00,NaN,1,1.31,NA,NA,2.09
100,50,0.50,0.18,0.85 ( 0.02 ),1.85 ( 0.11 ),4.11 ( 0.1 ),1.79 ( 0.01 ),3.74,0.48,2,0.85,1.85,4.11,1.79
500,50,0.10,0.80,0.47 ( 0.01 ),0.3 ( 0.05 ),1.61 ( 0.08 ),1.31 ( 0 ),4.69,0.05,3,0.47,0.30,1.61,1.31
1000,50,0.05,0.88,0.35 ( 0 ),0.07 ( 0.03 ),0.78 ( 0.07 ),1.17 ( 0 ),5.29,0.01,4,0.35,0.07,0.78,1.17
50,100,2.00,NaN,1.32 ( 0.05 ),1 ( 0 ),6 ( 0 ),2.23 ( 0.02 ),0.00,NaN,5,1.32,NA,NA,2.23
100,100,1.00,0.08,0.91 ( 0.03 ),4.33 ( 0.22 ),4.45 ( 0.09 ),2.02 ( 0.01 ),5.88,0.72,6,0.91,4.33,4.45,2.02
500,100,0.20,0.60,0.59 ( 0.01 ),1.91 ( 0.15 ),1.41 ( 0.07 ),1.54 ( 0 ),6.50,0.27,7,0.59,1.91,1.41,1.54
1000,100,0.10,0.81,0.47 ( 0 ),0.71 ( 0.09 ),0.53 ( 0.06 ),1.39 ( 0 ),6.18,0.1,8,0.47,0.71,0.53,1.39
50,500,10.00,NaN,1.33 ( 0.05 ),1 ( 0 ),6 ( 0 ),2.48 ( 0.03 ),0.00,NaN,9,1.33,NA,NA,2.48
100,500,5.00,0.02,1.07 ( 0.03 ),23.8 ( 0.52 ),4.26 ( 0.1 ),2.43 ( 0.02 ),25.54,0.93,10,1.07,23.80,4.26,2.43


In [36]:
## nothing was selected when Stab is NaN
for (i in c(1,5,9,13)){
    print(sum(results_ind_rf[[i]]$Stab.table))
}

[1] 0
[1] 0
[1] 0
[1] 0


In [37]:
## assign zero values to NA Stab; also recover missing FP values (1 in this case); missing FN values (6 in this case)
result.table_ind$Stab[is.na(result.table_ind$Stab)] = 0
result.table_ind$FP_mean[is.na(result.table_ind$FP_mean)] = 1
result.table_ind$FN_mean[is.na(result.table_ind$FN_mean)] = 6
#result.table_ind$FDR[is.na(result.table_ind$FDR)] = 0
result.table_ind

## export
write.table(result.table_ind, '../results_summary_cts/sim_ind_rf.txt', sep='\t', row.names=F)

N,P,Ratio,Stab,MSE,FP,FN,OOB,num_select,FDR,Index,MSE_mean,FP_mean,FN_mean,OOB_mean
50,50,1.00,0.00,1.31 ( 0.06 ),1 ( 0 ),6 ( 0 ),2.09 ( 0.02 ),0.00,NaN,1,1.31,1.00,6.00,2.09
100,50,0.50,0.18,0.85 ( 0.02 ),1.85 ( 0.11 ),4.11 ( 0.1 ),1.79 ( 0.01 ),3.74,0.48,2,0.85,1.85,4.11,1.79
500,50,0.10,0.80,0.47 ( 0.01 ),0.3 ( 0.05 ),1.61 ( 0.08 ),1.31 ( 0 ),4.69,0.05,3,0.47,0.30,1.61,1.31
1000,50,0.05,0.88,0.35 ( 0 ),0.07 ( 0.03 ),0.78 ( 0.07 ),1.17 ( 0 ),5.29,0.01,4,0.35,0.07,0.78,1.17
50,100,2.00,0.00,1.32 ( 0.05 ),1 ( 0 ),6 ( 0 ),2.23 ( 0.02 ),0.00,NaN,5,1.32,1.00,6.00,2.23
100,100,1.00,0.08,0.91 ( 0.03 ),4.33 ( 0.22 ),4.45 ( 0.09 ),2.02 ( 0.01 ),5.88,0.72,6,0.91,4.33,4.45,2.02
500,100,0.20,0.60,0.59 ( 0.01 ),1.91 ( 0.15 ),1.41 ( 0.07 ),1.54 ( 0 ),6.50,0.27,7,0.59,1.91,1.41,1.54
1000,100,0.10,0.81,0.47 ( 0 ),0.71 ( 0.09 ),0.53 ( 0.06 ),1.39 ( 0 ),6.18,0.1,8,0.47,0.71,0.53,1.39
50,500,10.00,0.00,1.33 ( 0.05 ),1 ( 0 ),6 ( 0 ),2.48 ( 0.03 ),0.00,NaN,9,1.33,1.00,6.00,2.48
100,500,5.00,0.02,1.07 ( 0.03 ),23.8 ( 0.52 ),4.26 ( 0.1 ),2.43 ( 0.02 ),25.54,0.93,10,1.07,23.80,4.26,2.43
